In [1]:
import pandas as pd
random_state = 20
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [2]:
def read_dataset(dataset, model_name):
    import pandas as pd
    import numpy as np
    import pickle
    from IPython.display import clear_output
    from sklearn import preprocessing
    clear_output()
    #datasets = ['TC02_20news', 'TC14_wos']
    #models_names = ["all-MiniLM-L6-v2","all-MiniLM-L12-v2"]
    if model_name != "TFIDF":
        embeddings_path = f'Arquivos TCC/embeddings/sbert/{model_name}_{dataset}.pkl'
        with open(embeddings_path, "rb") as fIn:
            stored_data = pickle.load(fIn)
        embeddings = stored_data['embeddings']
        labels = stored_data['labels']
        labels_enc = preprocessing.LabelEncoder().fit_transform(labels)
        unique, counts = np.unique(stored_data['labels'], return_counts=True)
        #Split hold-out test data
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
        return (X_train, X_test, y_train, y_test)
    else:
        df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
        df = df.dropna()
        textos = df['text'].values.tolist()
        labels = df['label']
        labels_enc = preprocessing.LabelEncoder().fit_transform(labels)
        #Split hold-out test data
        from sklearn.model_selection import train_test_split
        textos_train, textos_test, y_train, y_test = train_test_split(textos, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
        return (textos_train, textos_test, y_train, y_test)

def get_opt_params(pipeline):
    import ast
    df_cv = pd.read_csv('Arquivos TCC/perfomance_cv.csv')
    df_cv = df_cv.drop_duplicates(subset=['pipeline'], keep='last')
    params = df_cv.loc[df_cv['pipeline']==pipeline , 'params']
    params = ast.literal_eval(params.values[0])
    return params

def eval_classifier(pipeline):
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import LinearSVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import balanced_accuracy_score
    #pipeline = TC02_20news+all-MiniLM-L6-v2+KNN
    dataset, model_name ,classifier =tuple(pipeline.split('+'))
    params = get_opt_params(pipeline=pipeline) 
    print(params)
    X_train, X_test, y_train, y_test = read_dataset(dataset=dataset, model_name=model_name)
    if model_name != "TFIDF":
        if classifier == 'KNN':
            clf = KNeighborsClassifier(**params)
        if classifier == 'LR':
            clf = LogisticRegression(**params)
        if classifier == 'SVC':
            clf = LinearSVC(**params)
        if classifier == 'RF':
            clf = RandomForestClassifier(**params)
        clf.fit(X_train, y_train)
        balanced_acc = balanced_accuracy_score(y_test,clf.predict(X_test))
        print(color.BOLD+"A acurácia balanceada nos dados de teste: {0:.4f}".format(balanced_acc)+color.END)
        return balanced_acc
    else:
        if classifier == 'KNN':
            clf = KNeighborsClassifier()
        if classifier == 'LR':
            clf = LogisticRegression()
        if classifier == 'SVC':
            clf = LinearSVC()
        if classifier == 'RF':
            clf = RandomForestClassifier()
        pipeline_clf = make_pipeline(TfidfVectorizer(stop_words='english',smooth_idf=True), clf)
        pipeline_clf.set_params(**params)
        textos_train = X_train
        textos_test = X_test
        pipeline_clf.fit(textos_train, y_train)
        balanced_acc = balanced_accuracy_score(y_test,pipeline_clf.predict(textos_test))
        print(color.BOLD+"A acurácia balanceada nos dados de teste: {0:.4f}".format(balanced_acc)+color.END)
        return balanced_acc


In [3]:
df_cv = pd.read_csv('Arquivos TCC/perfomance_cv.csv')
df_cv = df_cv.drop_duplicates(subset=['pipeline'], keep='last')
for pipe in df_cv['pipeline'].unique()[-5:]:
    params = get_opt_params(pipeline=pipe)
    
type(params)

dict

In [4]:
dataset= 'TC02_20news'
model_name='TFIDF'
X_train, X_test, y_train, y_test = read_dataset(dataset=dataset, model_name=model_name)
X_train[0][:10]

'and key si'

In [5]:
df_cv = pd.read_csv('Arquivos TCC/perfomance_cv.csv')
df_cv = df_cv.drop_duplicates(subset=['pipeline'], keep='last')
#for pipe in df_cv['pipeline'].unique()[:1]:

pipe = 'TC14_wos+TFIDF+SVC'
print(pipe,eval_classifier(pipeline=pipe))

A acurácia balanceada nos dados de teste: 0.8229
TC14_wos+TFIDF+SVC 0.822927757798685
